# Merge por coordenadas con DuckDB

- Lee inmuebles con columnas latitud/longitud y colonia.
- Lee AGEB desde CSVs y asegura latitud/longitud en DOUBLE.
- Extrae y estandariza neighbourhood de AGEB.
- Join por colonia (estandarizada) y elige el AGEB más cercano dentro de la misma colonia.
- Devuelve df_merge y guarda parquet.

In [1]:
import os
import re
import duckdb
import pandas as pd
from unidecode import unidecode
from pyproj import Transformer
from datetime import datetime

# Parámetros de entrada/salida (dinámicos por fecha)
# "2025-08-31", "2025-09-14", "2025-09-20", "2025-09-27", "2025-10-05", "2025-10-11"
run_date = "2025-09-14"
run_date = datetime.strptime(run_date, "%Y-%m-%d").strftime("%Y-%m-%d")

inmuebles_path = f"../../data/processed/inmuebles24_departamentos_coordenadas_{run_date}.parquet"  # requiere latitud/longitud y colonia
ageb_input = "../../data/processed/INEGI/colonia/*.csv"  # CSVs de INEGI (glob) o ruta única
output_path = f"../../data/processed/merged_inmuebles24_departamentos_duckdb_{run_date}.parquet"

# Función para estandarizar texto
def estandarizar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = texto.replace("\n", " ")  # Reemplazar saltos de línea por espacios
    texto = texto.replace(",", "")  # Quitar comas
    return unidecode(texto.strip().lower())

In [47]:
import pandas as pd
import glob

# 1. Encontrar todos los paths que coinciden con el patrón
ruta_archivos = "../../data/processed/INEGI/colonia/*.csv"
lista_archivos = glob.glob(ruta_archivos)

# 2. Leer cada archivo CSV y almacenarlo en una lista de DataFrames
lista_dataframes = []
for nombre_archivo in lista_archivos:
    df_temporal = pd.read_csv(nombre_archivo, dtype = {'postcode':'string'})
    lista_dataframes.append(df_temporal)

# 3. Concatenar todos los DataFrames de la lista en uno solo
df_final = pd.concat(lista_dataframes, ignore_index=True)

/var/folders/c7/mdvpp7_n2dj49y88d6cj51kc0000gn/T/ipykernel_25707/418099082.py:11: DtypeWarning: Columns (50,111,112,116,117,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(nombre_archivo, dtype = {'postcode':'string'})
/var/folders/c7/mdvpp7_n2dj49y88d6cj51kc0000gn/T/ipykernel_25707/418099082.py:11: DtypeWarning: Columns (50,111,112,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(nombre_archivo, dtype = {'postcode':'string'})
/var/folders/c7/mdvpp7_n2dj49y88d6cj51kc0000gn/T/ipykernel_25707/418099082.py:11: DtypeWarning: Columns (50,111,112,116,117,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(nombre_archivo, dtype = {'postcode':'string'})
/var/folders/c7/mdvpp7_n2dj49y88d6cj51kc0000gn/T/ipykernel_25707/418099082.py:11: DtypeWarning: Columns (50,116,117,131) have mixed types. Specify dtype option on import 

In [43]:
df = pd.read_parquet(inmuebles_path)
df['cp'] = df['cp'].astype(str)
df['cp'] = df['cp'].str.zfill(5)

In [21]:
df.columns

Index(['precio_mxn', 'lote_m2', 'recamaras', 'baños', 'estacionamiento',
       'es_amueblado', 'es_penthouse', 'direccion', 'colonia', 'cp',
       'municipio', 'Título', 'Enlace', 'latitud', 'longitud'],
      dtype='object')

In [ ]:
df_final.neighbourhood.value_counts()

neighbourhood
                                            32008
agricola pantitlan                           2535
cabeza de juarez vi                          2047
zona escolar - arbolillo - jorge negrete     1778
jardin balbuena                              1573
                                            ...  
villa florence                                  1
lomas anahuac                                   1
san fernando                                    1
lomas del sol                                   1
san gabriel                                     1
Name: count, Length: 1873, dtype: int64

In [15]:
df_final['neighbourhood'] = df_final['address'].str.extract(pat)
df_final['neighbourhood'] = df_final['neighbourhood'].apply(estandarizar_texto)
df_final['neighbourhood'] = df_final['neighbourhood'].str.replace('colonia ', '').str.replace('Colonia ', '')


In [16]:
df_final.neighbourhood.value_counts()

neighbourhood
                                            32008
agricola pantitlan                           2535
cabeza de juarez vi                          2047
zona escolar - arbolillo - jorge negrete     1778
jardin balbuena                              1573
                                            ...  
villa florence                                  1
lomas anahuac                                   1
san fernando                                    1
lomas del sol                                   1
san gabriel                                     1
Name: count, Length: 1873, dtype: int64

In [25]:
df_final.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA',
       'POBTOT', 'POBFEM',
       ...
       'NOMREF3', 'lon', 'lat', 'municipio', 'address', 'road', 'quarter',
       'borough', 'postcode', 'neighbourhood'],
      dtype='object', length=135)

In [44]:
df.cp.isnull().sum()

0

In [45]:
cp_df = df.cp.unique().tolist()

In [46]:
cp_df.sort()
cp_df

['01000',
 '01010',
 '01020',
 '01030',
 '01040',
 '01050',
 '01060',
 '01070',
 '01080',
 '01090',
 '01100',
 '01120',
 '01140',
 '01150',
 '01160',
 '01180',
 '01210',
 '01250',
 '01260',
 '01269',
 '01270',
 '01279',
 '01280',
 '01290',
 '01330',
 '01410',
 '01420',
 '01430',
 '01460',
 '01470',
 '01480',
 '01500',
 '01510',
 '01530',
 '01550',
 '01588',
 '01590',
 '01600',
 '01700',
 '01710',
 '01729',
 '01740',
 '01760',
 '01770',
 '01780',
 '01800',
 '01810',
 '01830',
 '01840',
 '01900',
 '01904',
 '02000',
 '02010',
 '02020',
 '02040',
 '02050',
 '02060',
 '02080',
 '02090',
 '02120',
 '02130',
 '02140',
 '02150',
 '02160',
 '02230',
 '02240',
 '02250',
 '02300',
 '02360',
 '02440',
 '02459',
 '02460',
 '02480',
 '02490',
 '02519',
 '02530',
 '02600',
 '02630',
 '02670',
 '02680',
 '02700',
 '02710',
 '02720',
 '02730',
 '02790',
 '02800',
 '02810',
 '02830',
 '02840',
 '02900',
 '02930',
 '02940',
 '02950',
 '02980',
 '02990',
 '03000',
 '03010',
 '03020',
 '03023',
 '03100',


In [48]:
df_final.postcode.isnull().sum()    

11

In [49]:
cp_ageb = df_final.postcode.unique().tolist()

In [50]:
#cp_ageb.sort()
cp_ageb

['09100',
 '08100',
 '09210',
 '08500',
 '09010',
 '08510',
 '09030',
 '09220',
 '09040',
 '09310',
 '09208',
 '09130',
 '09230',
 '09140',
 '09430',
 '09438',
 '09429',
 '09410',
 '09420',
 '08900',
 '09300',
 '09320',
 '09530',
 '09239',
 '09440',
 '09479',
 '09450',
 '09400',
 '09810',
 '09340',
 '09200',
 '09240',
 '09470',
 '09460',
 '09060',
 '09070',
 '09000',
 '09520',
 '09510',
 '09089',
 '09180',
 '09480',
 '09080',
 '04230',
 '09099',
 '09090',
 '09360',
 '09350',
 '09359',
 '09209',
 '09260',
 '09570',
 '09500',
 '09560',
 '09820',
 '09828',
 '09849',
 '09829',
 '09860',
 '09250',
 '09270',
 '09280',
 '09700',
 '09290',
 '09620',
 '09600',
 '09630',
 '09819',
 '09800',
 '09840',
 '09830',
 '09858',
 '04260',
 '09850',
 '09710',
 '09720',
 '09750',
 '09550',
 '09740',
 '09870',
 '09890',
 '09770',
 '09868',
 '16036',
 '09880',
 '09780',
 '09900',
 '09790',
 '09760',
 '09769',
 '132219',
 '09920',
 '09910',
 '09970',
 '09930',
 '09960',
 '13230',
 '09020',
 '09609',
 '09837',

In [17]:
colonias = df_final.neighbourhood.unique().tolist()

In [19]:
[c for c in colonias if 'tlalpan' in c]

['vergel tlalpan',
 'tlalpan centro i',
 'guadalupe tlalpan',
 'hacienda de san juan de tlalpan 2a. seccion',
 'tlalpan centro ii',
 'torres tlalpan fovissste',
 'villa tlalpan',
 'bosque de tlalpan',
 'santiago tepalcatlalpan',
 'multifamiliar tlalpan',
 'rincon de tlalpan']

In [2]:
# Conexión DuckDB + extensión spatial
con = duckdb.connect(database=":memory:")
try:
    con.execute("INSTALL spatial;")
except Exception:
    pass
con.execute("LOAD spatial;")

# Cargar AGEB desde CSVs a pandas y transformar coordenadas (INEGI EPSG:6372 -> WGS84 EPSG:4326)
df_ageb = con.execute(f"""
SELECT *
FROM read_csv_auto('{ageb_input}', ignore_errors=True, all_varchar=True)
""").fetchdf()

if {'lon','lat'}.issubset(df_ageb.columns):
    # Convertir a float y transformar con pyproj
    df_ageb[['lon','lat']] = df_ageb[['lon','lat']].astype(float)
    transformer = Transformer.from_crs("epsg:6372", "epsg:4326", always_xy=True)
    lon_wgs, lat_wgs = transformer.transform(df_ageb['lon'].values, df_ageb['lat'].values)
    df_ageb['longitud'] = lon_wgs
    df_ageb['latitud'] = lat_wgs
elif {'longitud','latitud'}.issubset(df_ageb.columns):
    # Ya vienen en WGS84, asegurar tipo float
    df_ageb[['longitud','latitud']] = df_ageb[['longitud','latitud']].astype(float)
else:
    raise ValueError("AGEB necesita columnas lon/lat o longitud/latitud para la transformación.")

# Extraer y estandarizar neighbourhood de address
pat = re.compile(r"'neighbourhood':\s*'([^']+)'")
df_ageb['neighbourhood'] = df_ageb['address'].str.extract(pat)
df_ageb['neighbourhood'] = df_ageb['neighbourhood'].apply(estandarizar_texto)
df_ageb['neighbourhood'] = df_ageb['neighbourhood'].str.replace('colonia ', '').str.replace('Colonia ', '')

# Estandarizar secciones de colonias (e.g., 1a -> i)
replacements = {
    "1a": "i",
    "2a": "ii",
    "3a": "iii",
    "4a": "iv",
    "5a": "v"
}
for old, new in replacements.items():
    df_ageb['neighbourhood'] = df_ageb['neighbourhood'].str.replace(old, new, regex=False)

# Registrar tabla ageb en DuckDB
con.register("ageb_df", df_ageb)
con.execute("CREATE OR REPLACE TABLE ageb AS SELECT * FROM ageb_df")

# Cargar inmuebles y estandarizar colonia
df_inmuebles = pd.read_parquet(inmuebles_path)
df_inmuebles['colonia_std'] = df_inmuebles['colonia'].apply(estandarizar_texto)
con.register("inmuebles_df", df_inmuebles)
con.execute("CREATE OR REPLACE TABLE inmuebles AS SELECT * FROM inmuebles_df")

# Validar columnas de coordenadas en inmuebles
inm_cols = [r[1] for r in con.execute("PRAGMA table_info('inmuebles')").fetchall()]
if not {'latitud','longitud','colonia_std'}.issubset(set(inm_cols)):
    raise ValueError("El parquet de inmuebles debe incluir columnas 'latitud', 'longitud' y 'colonia'.")
con.execute("ALTER TABLE inmuebles ALTER COLUMN latitud TYPE DOUBLE")
con.execute("ALTER TABLE inmuebles ALTER COLUMN longitud TYPE DOUBLE")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
# Join por colonia estandarizada: toma el AGEB más cercano dentro de la misma colonia
query = """
WITH inmuebles_validos AS (
    SELECT * 
    FROM inmuebles
    WHERE latitud IS NOT NULL AND longitud IS NOT NULL
)
SELECT 
  i.*, 
  a.*, 
  ST_Distance(ST_Point(i.longitud, i.latitud), ST_Point(a.longitud, a.latitud)) AS distancia
FROM inmuebles_validos i
LEFT JOIN LATERAL (
  SELECT a.*
  FROM ageb a
  WHERE a.neighbourhood = i.colonia_std
  ORDER BY ST_Distance(ST_Point(i.longitud, i.latitud), ST_Point(a.longitud, a.latitud)) ASC
  LIMIT 1
) a ON TRUE
"""

df_merge = con.execute(query).fetchdf()
print("df_merge shape:", df_merge.shape)
df_merge.head(3)

df_merge shape: (6040, 154)


,precio_mxn,lote_m2,recamaras,baños,estacionamiento,es_amueblado,es_penthouse,direccion,colonia,cp,...,municipio_1,address,road,quarter,borough,postcode,longitud_1,latitud_1,neighbourhood,distancia
0,16000.0,30,1.0,1.0,1.0,1,0,av. tamaulipas 257 alvaro obregon ciudad de m...,alvaro obregon,9230,...,venustiano_carranza,"{'road': 'Avenida Congreso de la Unión', 'neig...",Avenida Congreso de la Unión,None,Venustiano Carranza,15990,-99.121470,19.412086,alvaro obregon,0.153109
1,50000.0,167,3.0,2.0,3.0,0,0,av bernardo quintana alvaro obregon ciudad de...,alvaro obregon,9230,...,venustiano_carranza,"{'road': 'Avenida Congreso de la Unión', 'neig...",Avenida Congreso de la Unión,None,Venustiano Carranza,15990,-99.121470,19.412086,alvaro obregon,0.137192
2,38000.0,80,2.0,1.0,1.0,0,0,zamora 70 - condesa cuauhtemoc,condesa,6140,...,cuauhtemoc,"{'house_number': '70', 'road': 'Calle Zamora',...",Calle Zamora,None,Cuauhtémoc,06140,-99.177612,19.416966,condesa,0.000157


In [4]:
df_merge.isnull().sum()

precio_mxn            0
lote_m2              50
recamaras             0
baños                 0
estacionamiento       0
                   ... 
postcode            741
longitud_1          741
latitud_1           741
neighbourhood       741
distancia          2130
Length: 154, dtype: int64

In [10]:
df_merge.AGEB.isnull().sum()

182

In [9]:
df_merge[df_merge.AGEB.isnull()].colonia.value_counts()

colonia
bosque                          38
tlalpan                         35
san angel                       18
del valle sur                   12
san jeronimo lidice             10
insurgentes cuicuilco            8
del valle norte                  7
vallejo                          5
pedregal de carrasco             5
ninos heroes                     4
moctezuma 2a seccion             3
ciudad de los deportes           3
san mateo                        3
arenal                           3
18                               2
san lucas                        2
ampliacion alpes                 2
pedregal de santa ursula         2
la otra banda                    2
los girasoles                    2
juan escutia                     1
la magdalena                     1
culhuacan                        1
san martin xochinahuac           1
del nino jesus                   1
lomas de becerra                 1
villa coyoacan                   1
santiago atzacoalco              1
infonavit iz

In [5]:
# Guardar parquet final
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_merge.to_parquet(output_path, index=False)
print("Parquet escrito en:", output_path)

Parquet escrito en: ../../data/processed/merged_inmuebles24_departamentos_duckdb_2025-09-14.parquet
